<a href="https://colab.research.google.com/github/LisburnLad/Kaggle/blob/master/Copy_of_Energy2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

Mounted at /content/gdrive


In [0]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
import gc
from fastai.vision import *

In [0]:
google_drive = Path(base_dir)

In [0]:
# !rm 'energy_utils.py'
# !cp '/content/gdrive/My Drive/fastai-v3/energy_utils.py' .
# from energy_utils import *

In [0]:
import zipfile
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()


# unzip the specified file and delete the zip file on completion
def unzip_and_delete( aPath, aSourceZip ):
  with zipfile.ZipFile(str(aPath/aSourceZip), 'r') as zip_ref:
      zip_ref.extractall(str(aPath))  
  os.remove(str(aPath/aSourceZip))     

# generate additional fields from the date field
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    # targ_pre = ""
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)   
  

#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist  


def create_dataframe( path_energy, aMainZip, aWeatherZip ):

  unzip_and_delete( path_energy, aMainZip )
  unzip_and_delete( path_energy, aWeatherZip )

  building_df = pd.read_csv(path_energy/"building_metadata.csv")
  weather_train = pd.read_csv(path_energy/"weather_train.csv")
  main_df = pd.read_csv(path_energy/"train.csv")   

  main_df = main_df.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
  main_df = main_df.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"]) 
  
  del building_df
  del weather_train

  # do memory reduction on main columns
  main_df, NAlist = reduce_mem_usage(main_df)

  # copy the timestamp column to make a new date column
  # (easier for the 'add_datepart' function to work with)
  main_df['date'] = main_df['timestamp']

  # generate extra fields based on the date and time
  add_datepart(main_df, "date", drop=False, time=True)

  # drop minutes and second as they are all zero
  main_df.drop('Minute', axis=1, inplace=True)   
  main_df.drop('Second', axis=1, inplace=True)    

  # drop the date column to make a new one
  main_df.drop('date', axis=1, inplace=True)    

  # extract the date from the timestamp
  main_df['date'] = main_df.timestamp.str.split(' ', expand=True)[0]  

  # drop the timestamp column as its now covered by date etc.
  main_df.drop('timestamp', axis=1, inplace=True)    

  # change the column types to make them have the lowest memory footprint
  main_df, NAlist = reduce_mem_usage(main_df)
 
  return main_df    

In [0]:
path_energy = Config.data_path()/'energy'
path_energy.mkdir(parents=True, exist_ok=True) 

!mkdir -p ~/.kaggle/
!cp '/content/gdrive/My Drive/fastai-v3/kaggle.json' ~/.kaggle/    

!pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [0]:
energy_gdrive_dir = Path(base_dir + 'Energy')
os.path.exists(energy_gdrive_dir/'training.pkl')

True

In [0]:
# test if the preprocessed data already exists
train_exists = os.path.exists(energy_gdrive_dir/'training.pkl')
test_exists = os.path.exists(energy_gdrive_dir/'testing.pkl')
if (not train_exists or not test_exists):  

  # download the data from Kaggle
  !kaggle competitions download -c ashrae-energy-prediction -p {path_energy} 

  print("Generating test data file")

  # preprocess the data and save to google drive
  test = create_dataframe( path_energy, "test.csv.zip", "weather_test.csv.zip" )  
  test.to_pickle(energy_gdrive_dir/'testing.pkl') 

  # delete the test set for now or we'll run out of memory
  del test  
  gc.collect()  


  print("Generating training data file")

  # preprocess the data and save to google drive
  train = create_dataframe( path_energy, "train.csv.zip", "weather_train.csv.zip" )  
  train.to_pickle(energy_gdrive_dir/'training.pkl')  

else:

  # read cleaned data from pickle file
  print("Reading preprocessed data")
  train = pd.read_pickle(energy_gdrive_dir/'training.pkl')    

Reading preprocessed data


In [0]:
train.head(5)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Hour,Elapsed,date
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008,0,25.0,6,20.0,-2,1019.700012,0,0.0,2016,1,53,1,4,1,0,1,0,1,0,1,0,1451606400,2016-01-01
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004,0,25.0,6,20.0,-2,1019.700012,0,0.0,2016,1,53,1,4,1,0,1,0,1,0,1,0,1451606400,2016-01-01
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991,0,25.0,6,20.0,-2,1019.700012,0,0.0,2016,1,53,1,4,1,0,1,0,1,0,1,0,1451606400,2016-01-01
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002,0,25.0,6,20.0,-2,1019.700012,0,0.0,2016,1,53,1,4,1,0,1,0,1,0,1,0,1451606400,2016-01-01
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975,0,25.0,6,20.0,-2,1019.700012,0,0.0,2016,1,53,1,4,1,0,1,0,1,0,1,0,1451606400,2016-01-01


In [0]:
# building_df.columns

# Index(['site_id', 'building_id', 'primary_use', 'square_feet', 'year_built',
#        'floor_count'],
#       dtype='object')

In [0]:
# weather_train.columns

# Index(['site_id', 'timestamp', 'air_temperature', 'cloud_coverage',
#        'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
#        'wind_direction', 'wind_speed'],
#       dtype='object')

In [0]:
# test for Null values after the join from the building_df
len(train[train.primary_use.isnull()]),len(train[train.year_built.isnull()])

(0, 0)

In [0]:
# test for Null values after the join from the weather_df
len(train[train.air_temperature.isnull()]),len(train[train.cloud_coverage.isnull()]),len(train[train.wind_speed.isnull()])

(0, 0, 0)

In [0]:
train.isnull().sum()

building_id           0
meter                 0
timestamp             0
meter_reading         0
site_id               0
primary_use           0
square_feet           0
year_built            0
floor_count           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
Year                  0
Month                 0
Week                  0
Day                   0
Dayofweek             0
Dayofyear             0
Is_month_end          0
Is_month_start        0
Is_quarter_end        0
Is_quarter_start      0
Is_year_end           0
Is_year_start         0
Hour                  0
Elapsed               0
date                  0
dtype: int64

In [0]:
# how many sites are there:
len(train.site_id.unique())

16

In [0]:
# how can there be 19 unique floor counts when there are only 16 sites?
# - because each site can have more than one building
len(train.floor_count.unique())

19

In [0]:
train.site_id.unique()

array([ 0,  1,  2,  3,  7,  8, 11, 12, 13,  4,  5,  6,  9, 10, 14, 15], dtype=uint64)

In [0]:
for site in train.site_id.unique():  
  # print(site,":",train.loc[train.site_id==site,"floor_count"].unique())
  print(site,":",train.loc[train.site_id==site,"building_id"].unique())

0 : [  0   1   2   3 ... 103 104  29  48]
1 : [105 106 107 108 ... 152 153 154 155]
2 : [156 157 158 159 ... 287 288 289 290]
3 : [291 292 293 294 ... 299 528 555 403]
7 : [789 790 791 792 793 794 795 796 797 798 799 800 801 802 803]
8 : [804 805 806 807 ... 872 873 829 839]
11 : [1029 1030 1031 1032 1028]
12 : [1033 1034 1035 1036 1037 1038 1039 1040 1041 1042 1043 1044 1045 1046 1047 1048 1049 1050 1051 1052 1053 1054 1055
 1056 1057 1058 1059 1060 1061 1062 1063 1064 1065 1066 1067 1068]
13 : [1069 1070 1071 1072 ... 1219 1220 1221 1222]
4 : [565 566 569 570 ... 568 567 621 591]
5 : [656 657 658 659 ... 741 742 743 744]
6 : [745 746 747 748 749 750 751 753 754 755 756 757 758 759 760 761 762 764 765 766 767 768 769 770 771 772 773 774 775
 776 777 778 779 780 781 782 784 785 787 788 752 763 786 783]
9 : [874 875 876 877 ... 996 997 943 909]
10 : [ 998  999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020
 1021 1022 1023 1024 10

In [0]:
train.Hour.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], dtype=uint64)

In [0]:
# how many date readings are there for site_id 0
len(train.loc[train.site_id==0,"date"].unique())

366

In [0]:
# arrange the data from oldest to newest
train = train.sort_values(['building_id', 'date', 'Hour'], ascending=[True, True, True])

In [0]:
train.head(5)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Hour,Elapsed,date
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008,0,25.000000,6,20.0,-2,1019.700012,0,0.0,2016,1,53,1,4,1,0,1,0,1,0,1,0,1451606400,2016-01-01
1187,0,0,2016-01-01 01:00:00,0.0,0,Education,7432,2008,0,24.400000,255,21.1,-1,1020.200012,70,1.5,2016,1,53,1,4,1,0,1,0,1,0,1,1,1451610000,2016-01-01
3480,0,0,2016-01-01 02:00:00,0.0,0,Education,7432,2008,0,22.799999,2,21.1,0,1020.200012,0,0.0,2016,1,53,1,4,1,0,1,0,1,0,1,2,1451613600,2016-01-01
5779,0,0,2016-01-01 03:00:00,0.0,0,Education,7432,2008,0,21.100000,2,20.6,0,1020.099976,0,0.0,2016,1,53,1,4,1,0,1,0,1,0,1,3,1451617200,2016-01-01
8075,0,0,2016-01-01 04:00:00,0.0,0,Education,7432,2008,0,20.000000,2,20.0,-1,1020.000000,250,2.6,2016,1,53,1,4,1,0,1,0,1,0,1,4,1451620800,2016-01-01


In [0]:
from fastai.tabular import *

In [0]:
# drop the timestamp column as its now covered by date etc.
if 'timestamp' in train:
  train.drop('timestamp', axis=1, inplace=True)  

train.columns

Index(['building_id', 'meter', 'meter_reading', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed', 'Year', 'Month',
       'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end',
       'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
       'Is_year_start', 'Hour', 'Elapsed', 'date'],
      dtype='object')

In [0]:
train.meter_reading.describe()

count    2.012560e+07
mean     2.050410e+03
std      1.535616e+05
min      0.000000e+00
25%      1.830000e+01
50%      7.875000e+01
75%      2.678430e+02
max      2.190470e+07
Name: meter_reading, dtype: float64

In [0]:
train = train.set_index("Date")

In [0]:
columns = ['building_id', 'air_temperature', 'precip_depth_1_hr', 'cloud_coverage', 'dew_temperature']
df = train[columns]
# df = df.set_index('date')
# df.set_index('date', inplace=True, drop=False)
df.head(10)

,building_id,air_temperature,precip_depth_1_hr,cloud_coverage,dew_temperature
0,0,25.000000,-2,6,20.0
1187,0,24.400000,-1,255,21.1
3480,0,22.799999,0,2,21.1
5779,0,21.100000,0,2,20.6
8075,0,20.000000,-1,2,20.0
10134,0,19.400000,0,255,19.4
12191,0,21.100000,-1,6,21.1
14247,0,21.100000,0,255,21.1
16304,0,20.600000,0,255,20.0
18361,0,21.100000,0,255,20.6


In [0]:
bwd = df[columns].groupby("building_id").rolling(7, min_periods=1).sum()

In [0]:
bwd.head(10)

building_id  ...  dew_temperature
building_id                     ...                 
0           0              0.0  ...        20.000000
            1187           0.0  ...        41.100000
            3480           0.0  ...        62.200001
            5779           0.0  ...        82.800001
            8075           0.0  ...       102.800001
            10134          0.0  ...       122.200001
            12191          0.0  ...       143.300001
            14247          0.0  ...       144.400002
            16304          0.0  ...       143.300001
            18361          0.0  ...       142.800001

[10 rows x 5 columns]

In [0]:
# fwd = df[['Store']+columns].sort_index(ascending=False
#                                       ).groupby("Store").rolling(7, min_periods=1).sum()

# Model Training

In [0]:
# cat_names = ["building_id", "meter", "site_id", "primary_use", "floor_count", "hour", "day", "weekday", "month", "meter"]
# cont_names = ["square_feet", "year_built", "air_temperature", "cloud_coverage", "dew_temperature"]
# drop_cols = ["precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]

cat_names = [ 'building_id', 'meter', 'site_id', 'primary_use', 'floor_count', 'Year', 'Month',
              'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
              'Is_year_start', 'Hour', 'Elapsed' ]

cont_names = [ 'square_feet', 'year_built', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
               'sea_level_pressure', 'wind_direction', 'wind_speed' ]

drop_cols = [ 'date' ]

dep_var = 'meter_reading'

procs = [FillMissing, Categorify, Normalize]

In [0]:
data = (TabularList.from_df(train, path=path_energy, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))                           
                           .label_from_df(cols=dep_var, label_cls=FloatList, log=True)                          
                           .databunch())

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: RuntimeWarning: divide by zero encountered in log
  return FloatItem(np.log(o) if self.log else o)


In [0]:
data.show_batch(rows=10)

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: RuntimeWarning: divide by zero encountered in log
  return FloatItem(np.log(o) if self.log else o)


building_id,meter,site_id,primary_use,floor_count,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Hour,Elapsed,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,target
596,0,4,Education,4,2016,8,32,13,5,226,0,0,0,0,0,0,19,1471114800,-0.7623,-0.0926,0.3130,-0.8748,0.5464,-0.0421,0.9716,-0.2631,0.0425,2.014903
1256,2,14,Office,0,2016,10,39,1,5,275,0,1,0,1,0,0,6,1475301600,-0.5963,-0.7091,-0.2529,1.1418,0.4376,-0.0421,1.0305,-0.2766,0.9657,5.0732155
958,2,9,Entertainment/public assembly,0,2016,5,18,2,0,123,0,0,0,0,0,0,4,1462161600,1.0843,-0.7091,0.5264,1.1418,0.5464,-0.0421,0.9691,-0.2583,-0.1773,3.948355
1429,0,15,Education,0,2016,11,46,15,1,320,0,0,0,0,0,0,23,1479250800,-0.5746,1.9367,-0.7630,1.1418,-0.4925,-0.1883,0.9384,-0.2595,-0.8807,3.6041381
238,1,2,Office,0,2016,11,47,26,5,331,0,0,0,0,0,0,13,1480165200,-0.5357,-0.7091,0.0625,-0.8589,-0.8289,-0.0421,0.9742,-0.2741,0.0425,4.3158026
778,1,6,Entertainment/public assembly,0,2016,4,14,7,3,98,0,0,0,0,0,0,12,1460030400,0.0047,-0.7091,-0.8187,1.1418,-0.1660,8.8739,0.8504,-0.2766,-0.6169,-inf
150,0,1,Office,8,2016,10,42,20,3,294,0,0,0,0,0,0,20,1476993600,-0.3596,0.7294,-0.5868,1.1418,-0.0373,-0.3344,1.0212,-0.2595,-0.3971,6.0275555
429,0,3,Public services,0,2016,1,4,27,2,27,0,0,0,0,0,0,2,1453860000,-0.8181,0.0872,-0.8651,1.1418,-0.6013,-0.0421,0.9759,-0.2686,0.0425,2.4510052
1407,1,15,Lodging/residential,0,2016,8,33,20,5,233,0,0,0,0,0,0,8,1471680000,-0.2103,2.0394,0.0069,-0.8908,0.6552,-0.3344,0.9588,-0.2796,-1.5402,5.0891447
246,0,2,Office,0,2016,4,15,17,6,108,0,0,0,0,0,0,16,1460908800,0.8079,1.0376,0.5264,-0.8748,-1.0465,-0.0421,0.9631,-0.2759,0.7019,5.8795547


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [0]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,nan,nan,0.000000,3:47:39


/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: RuntimeWarning: divide by zero encountered in log
  return FloatItem(np.log(o) if self.log else o)
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: RuntimeWarning: divide by zero encountered in log
  return FloatItem(np.log(o) if self.log else o)
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: RuntimeWarning: divide by zero encountered in log
  return FloatItem(np.log(o) if self.log else o)
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: RuntimeWarning: divide by zero encountered in log
  return FloatItem(np.log(o) if self.log else o)
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: RuntimeWarning: divide by zero encountered in log
  return FloatItem(np.log(o) if self.log else o)
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: RuntimeWarning: divide by zero encountered in log
  return FloatItem(np.log(o) if self.log else o)
/usr/local/lib/p

In [0]:
learn.save(google_drive/'energy-2')
learn.export(google_drive/'energy-2.pkl')

In [0]:
import gc
del train

gc.collect()

In [0]:
test_df = create_dataframe( path_energy, "test.csv.zip", "weather_test.csv.zip" ):

In [0]:
test_image_list = TabularList.from_df(test_df, path=path_energy, cat_names=cat_names, cont_names=cont_names, procs=procs)


In [0]:
learn = load_learner(google_drive/'energy-2.pkl')

In [0]:
learn.data.add_test(test_image_list)
preds,_ = learn.get_preds(DatasetType.Test)

In [0]:
res = np.concatenate(preds)

In [0]:
pd.DataFrame(res).describe()

In [0]:
res.shape

In [0]:
unzip_and_delete( path_energy, "sample_submission.csv.zip" )

In [0]:
sub = pd.read_csv(path_energy/"sample_submission.csv")

In [0]:
sub["meter_reading"] = res

In [0]:
sub.to_csv(path_energy/"submission.csv", index = False)

In [0]:
!kaggle competitions submit ashrae-energy-prediction -f {path_energy/'submission.csv'} -m "energy 2"